In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models, Sequential
from tensorflow.keras.optimizers import SGD, Adam, Lion
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Dropout, Conv3D, BatchNormalization, MaxPooling3D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [2]:
def create_c3d(input_shape=(16, 64, 64, 3), n_classes=2):
    model = Sequential(
        [
        Conv3D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling3D(pool_size=2),
    
        Conv3D(filters=64, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling3D(pool_size=2),
    
        Conv3D(filters=128, kernel_size=3, activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling3D(pool_size=2),
    
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(n_classes, activation='softmax')
        ]
    )

    return model

In [3]:
features_np = np.load(r"/kaggle/input/video-feature-np/features.npy")
labels = np.load(r"/kaggle/input/video-feature-np/labels.npy")
# video_files_paths = np.load(r"/kaggle/input/video-feature-np/video_files_paths.npy")

In [4]:
# convert labels into one-hot vectors
encoded_labels = to_categorical(labels)

In [5]:
x_train, x_temp, y_train, y_temp = train_test_split(features_np, 
                                                    encoded_labels,
                                                    stratify=encoded_labels,
                                                    test_size=0.3,
                                                    shuffle=True, 
                                                    random_state=2)

x_test, x_val, y_test, y_val = train_test_split(x_temp, 
                                                y_temp, 
                                                stratify=y_temp,
                                                test_size=0.5,
                                                shuffle=True, 
                                                random_state=2)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(1400, 16, 64, 64, 3)
(1400, 2)
(300, 16, 64, 64, 3)
(300, 2)
(300, 16, 64, 64, 3)
(300, 2)


In [25]:
early_stopping = EarlyStopping(monitor="val_accuracy", 
                                patience=10, 
                                restore_best_weights=True)

model = create_c3d()

model.compile(loss="categorical_crossentropy",
               optimizer=Lion(learning_rate=0.1), 
               metrics=["accuracy"])

model.fit(x_train, y_train, 
          epochs=100,
          batch_size=8,
          shuffle=True,
          validation_data=(x_val, y_val),
          callbacks=[early_stopping])

Epoch 1/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - accuracy: 0.4818 - loss: 2533.2776 - val_accuracy: 0.5067 - val_loss: 23914296.0000
Epoch 2/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.5473 - loss: 11556.2383 - val_accuracy: 0.5133 - val_loss: 2101052.0000
Epoch 3/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.5087 - loss: 3991.3860 - val_accuracy: 0.4333 - val_loss: 296645056.0000
Epoch 4/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.4613 - loss: 9452.8896 - val_accuracy: 0.5000 - val_loss: 39839028.0000
Epoch 5/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.5049 - loss: 0.7122 - val_accuracy: 0.4867 - val_loss: 14961108992.0000
Epoch 6/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.4875 - loss: 70811.1094 - val_accuracy: 0.5033 - val_loss: 3552761085952.0000
Epoch 7/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.5010 - loss: 0.7061 - val_accuracy: 0.4967 - val_loss: 33568298369024.0000
Epoch 8/1

In [26]:
acc = model.evaluate(x_test, y_test)[1]
print(f"{acc:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.5398 - loss: 614032.9375
0.5100


In [8]:
model.save("c3d.keras")